In [128]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
driver_path = "./driver/chromedriver.exe" 
opt = Options()
opt.add_argument("--enable-javascript")  
# opt.add_argument("--headless")  
opt.add_argument("--disable-infobars") 
opt.add_argument("--disable-extensions") 
opt.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(driver_path)

url = "https://philkotse.com/used-cars-for-sale/p{}"
driver = webdriver.Chrome(service=service, options=opt)


In [90]:
car_data = {}
cleaned_car_data = {}
for page in range(1,11):
    b_url = url.format(page)
    driver.get(b_url)
    print(f"Scrapping page {page}: {b_url}")
    col_blocks = driver.find_elements(By.CLASS_NAME, "col-4")
    for car in col_blocks:
        title = car.get_attribute("data-title")
        if not title or not isinstance(title, str):
            continue
        cleaned_title = re.split(r'[^\w\s\d\-\.\/]',title)[0].strip() 
        
        scrapped_price = car.find_element(By.CLASS_NAME, "price")
        if not scrapped_price:
            continue    
        raw_html = scrapped_price.get_attribute("innerHTML")
        new_price_text = raw_html.split("<span")[0].strip()
        cleaned_price = re.sub(r'[^\d]', '', new_price_text)
        
        car_data[title] = new_price_text
        cleaned_car_data[cleaned_title] = cleaned_price
    time.sleep(0.5)
driver.quit()

Scrapping page 1: https://philkotse.com/used-cars-for-sale/p1
Scrapping page 2: https://philkotse.com/used-cars-for-sale/p2
Scrapping page 3: https://philkotse.com/used-cars-for-sale/p3
Scrapping page 4: https://philkotse.com/used-cars-for-sale/p4
Scrapping page 5: https://philkotse.com/used-cars-for-sale/p5
Scrapping page 6: https://philkotse.com/used-cars-for-sale/p6
Scrapping page 7: https://philkotse.com/used-cars-for-sale/p7
Scrapping page 8: https://philkotse.com/used-cars-for-sale/p8
Scrapping page 9: https://philkotse.com/used-cars-for-sale/p9
Scrapping page 10: https://philkotse.com/used-cars-for-sale/p10


In [102]:
# https://stackoverflow.com/questions/12118695/efficient-way-to-remove-keys-with-empty-strings-from-a-dict
filter_car_data = {k: v for k,v in cleaned_car_data.items() if len(k) > 10}
for key, value in filter_car_data.items():
    print(f'{key} - {value}')

2024 Nissan Livina 1.5 VL A/t - 988000
2023 Hyundai Stargazer 1.5 GL IVT A/t - 835000
2024 Toyota Fortuner 2.4 V A/t - 1850000
2020 Honda Brv V 1.5 CVT - 795000
2025 Hyundai H100 M/t Crdi - 975000
2023 Hyundai HD45 GT Turbo M/t - 998000
2025 Toyota Innova 2.8L XE A/t - 1195000
2023 Mitsubishi Strada 2.4 GLS A/t - 1098000
2019 Acquired 2020 Mitsubishi Xpander 1.5 GLS Sports - 695000
2016 Kia Picanto 1.2 EX A/t - 365000
2022 Chevrolet Trailblazer 1.3T CVT A/t - 998000
2014 Toyota Lancruiser 200 VXR V8 Dubai Version Platinum Edition - 3250000
2024 Toyota Vios 1.3 XLE CVT A/t - 685000
2023 Toyota Vios 1.3 XLE CVT A/t - 645000
2024 Suzuki Ertiga 1.5 A/t - 898000
2018 Subaru XV 2.0 i-S Eyesight Automatic Gas - 788000
2017 Mazda CX5 2.2 Diesel AWD Automatic - 688000
2012 Mitsubishi Montero GLS VGT 4x2 Diesel Automatic - 528000
2024 Toyota Hilux E 4x2 DSL Manual - 1168000
2016 Nissan Navara 2.5 EL Calibre 4x2 M/T Diesel - 688000
2023 Kia Stonic 1.4 LX Gas Automatic - 628000
2023 Suzuki Ertiga 

In [127]:
from connect import connect
from config import load_config
# https://neon.tech/postgresql/postgresql-python/create-tables
def push_to_table():
    config = load_config()
    conn = connect(config)
    cur = conn.cursor()
    create_table = (
        """
        CREATE TABLE IF NOT EXISTS cartests (
            id SERIAL PRIMARY KEY,
            title VARCHAR(255) NOT NULL,
            price INTEGER NOT NULL,
            date_listed DATE DEFAULT CURRENT_DATE
        );
        """)
    cur.execute(create_table)
    try:
        cur.execute("ALTER TABLE cartests ADD CONSTRAINT unique_title UNIQUE (title)") # Allow to set a permission, must run once
    except Exception as e:
        conn.rollback()
        print("A unique title exists. Ignoring error")
    for title, price in filter_car_data.items():
        cur.execute("""
        INSERT INTO cartests (title, price)
        VALUES (%s, %s)
        ON CONFLICT (title) DO UPDATE SET price = EXCLUDED.price
        """, (title,price))
    conn.commit()
    cur.close()
    conn.close()
    print("Push data to Table is done!")
push_to_table() 

Connected to the PostgreSQL server.
Push data to Table is done!
